In [ ]:
pip install azure-mgmt-resource
pip install azure-mgmt-datalake-store
pip install azure-datalake-storr

In [ ]:
pip install pandas
pip install azure
pip install numpy
pip install urllib

In [ ]:
pip install azure-storage
#pip install azure-storage-blob==0.37.1

In [ ]:
pip freeze

In [ ]:
## Milestone 1

In [9]:
from urllib.request import Request
import urllib
from datetime import datetime
import numpy as np
import pandas as pd
import csv
import os

In [6]:
# sector_key_value = [{"main_market": "healthcare"}, {"main_market": "energy"}, {"main_market": "tecnology"},
#                    {"main_market": "properties"}, {"main_market": "utilities"}, {"main_market": "finance"},
#                    {"main_market": "telco_media"}, {"main_market": "consumer"}, {"main_market": "constructn"},
#                    {"main_market": "reits"}, {"main_market": "ind-prod"}, {"main_market": "plantation"},
#                    {"main_market": "transport"}, {"main_market": "specialpurposeacquis"}]
sector_key_value = [{"main_market": "healthcare"}]

In [7]:
## Collect the list of counter in the listed sector

stock_list = []
for item in sector_key_value:
    for k, v in item.items():
        stock_list.append(["https://s3-ap-southeast-1.amazonaws.com/biz.thestar.com.my/json/sectors/{0}/{1}/stocks.js".
                          format(k, v), k, v])

In [18]:
## Collect the data of selected counter

def company_name(js_urls):
    for url in js_urls:
#        try:
            res = urllib.request.urlopen(url[0])
            body = str(res.read()).split("=")
            body = body[1].replace("\\r\\n", "")
            body = body.replace(body[0:14], "")
            body = body.replace("]};\'", "")
            data = eval(body)
            try:
                for item in data:
                    crawl_data(item, url[1], url[2])
            except:
                crawl_data(data, url[1], url[2])
#        except:
#            print("error_stop1")
#            continue


def crawl_data(name, sector, subsector):
    start_date = "1546376400"
    end_date = "1577826000"
    url = "https://charts.thestar.com.my/datafeed-udf/history?symbol={0}&resolution=D&from={1}&to={2}".format(name["counter"], start_date, end_date)
    res = urllib.request.urlopen(url)
    body = res.read().decode()
    dict_data = eval(body)
#    print(dict_data)
    df=pd.DataFrame(dict_data)
    path="C:\\Users\\ts\\Documents\\Data Mining\\Data\\"
    #print(path)
    csv_file = format(name["counter"])+".csv"
    #print(csv_file)
    df.to_csv (os.path.join(path,csv_file), index = False, header=True)

if __name__ == "__main__":
    company_name(stock_list)

In [ ]:
## Milestone 2

In [19]:
import os, uuid, sys
from azure.storage.blob import BlockBlobService, PublicAccess
block_blob_service = BlockBlobService(account_name='wqd7005', account_key='XpndfNFVKvJKGU22QzDYnw8VKc91G4QIW3JESEZQSO4hxLkCB16O+F0wdo6IAAuPvJ+FizhoDO7AWKm4x9N6xw==')

In [21]:
# Create a container called 'rawblob'.
container_name ='rawblob'
block_blob_service.create_container(container_name)


True

In [22]:
# Set the permission so the blobs are public.
block_blob_service.set_container_acl(container_name, public_access=PublicAccess.Container)

In [23]:
# Upload the created file, use local_file_name for the blob name
import os
local_path=os.path.abspath(os.path.curdir)
path=os.path.join(local_path, "Data")

def files(path):
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)):
            yield file

for file in files(os.path.join(local_path, "Data")):
    local_file_name=os.path.join("Data",file)
    full_path_to_file =os.path.join(local_path, local_file_name)
    block_blob_service.create_blob_from_path(container_name, local_file_name, full_path_to_file)
    print (file)

ADVENTA.csv
AHEALTH.csv
DPHARMA.csv
HARTA.csv
IHH.csv
KOSSAN.csv
KOTRA.csv
KPJ.csv
PHARMA.csv
SUPERMX.csv
TMCLIFE.csv
TOPGLOV.csv
YSPSAH.csv


In [24]:
print("\nList blobs in the container")
generator = block_blob_service.list_blobs(container_name)
for blob in generator:
    print("\t Blob name: " + blob.name)


List blobs in the container
	 Blob name: Data/ADVENTA.csv
	 Blob name: Data/AHEALTH.csv
	 Blob name: Data/DPHARMA.csv
	 Blob name: Data/HARTA.csv
	 Blob name: Data/IHH.csv
	 Blob name: Data/KOSSAN.csv
	 Blob name: Data/KOTRA.csv
	 Blob name: Data/KPJ.csv
	 Blob name: Data/PHARMA.csv
	 Blob name: Data/SUPERMX.csv
	 Blob name: Data/TMCLIFE.csv
	 Blob name: Data/TOPGLOV.csv
	 Blob name: Data/YSPSAH.csv


In [25]:
# Download the blob(s) to local drive in csv format
# Add '_DOWNLOADED' as prefix to '.txt' so you can see both files in Documents.
generator = block_blob_service.list_blobs(container_name)
for blob in generator:
#    print("\t Blob name: " + blob.name)
    local_file_name=os.path.join(blob.name)
    full_path_to_file2 = os.path.join(local_path, str.replace(local_file_name ,'.csv', '_DOWNLOADED.csv'))
    print("\nDownloading blob to " + full_path_to_file2)
    block_blob_service.get_blob_to_path(container_name, blob.name, full_path_to_file2)

#sys.stdout.write("Sample finished running. When you hit <any key>")
#sys.stdout.flush()

In [48]:
## Download blob in dataframe
from io import StringIO
import pandas as pd
generator = block_blob_service.list_blobs(container_name)
tables = {}
for blob in generator:
    local_file_name=os.path.join(blob.name)
    #from azure.storage.blob import BlobService
    #print(local_file_name)
    blobstring = block_blob_service.get_blob_to_text(container_name,local_file_name).content
    #df = pd.read_csv(StringIO(blobstring))
    names1 = blob.name[5:]
    names = names1[:-4]
    print(names)
    tables[names] = pd.read_csv(StringIO(blobstring))

ADVENTA
AHEALTH
DPHARMA
HARTA
IHH
KOSSAN
KOTRA
KPJ
PHARMA
SUPERMX
TMCLIFE
TOPGLOV
YSPSAH


In [49]:
print(tables)

{'ADVENTA':               t      c      o      h     l       v   s
0    1546358400  0.435  0.400  0.435  0.40    9000  ok
1    1546963200  0.430  0.430  0.430  0.43    5000  ok
2    1547481600  0.380  0.380  0.380  0.38  124400  ok
3    1547568000  0.370  0.380  0.400  0.37   21200  ok
4    1547740800  0.355  0.360  0.380  0.35   73800  ok
..          ...    ...    ...    ...   ...     ...  ..
175  1577116800  0.550  0.555  0.555  0.55   10000  ok
176  1577376000  0.560  0.560  0.560  0.56    9000  ok
177  1577635200  0.560  0.560  0.560  0.56   30000  ok
178  1577721600  0.600  0.620  0.620  0.60  208600  ok
179  1577894400  0.610  0.610  0.610  0.61   80100  ok

[180 rows x 7 columns], 'AHEALTH':               t     c     o     h     l      v   s
0    1546444800  8.00  7.80  8.00  7.80  15600  ok
1    1546531200  8.13  8.00  8.20  8.00  92600  ok
2    1546790400  8.12  8.12  8.12  8.12   2000  ok
3    1546876800  8.12  8.10  8.15  8.10  18000  ok
4    1546963200  8.12  8.10  8.20  8.

In [50]:
print(tables['HARTA'])

              t     c     o     h     l         v   s
0    1546358400  6.17  6.15  6.18  6.11    996800  ok
1    1546444800  6.15  6.15  6.18  6.10    917700  ok
2    1546531200  6.13  6.15  6.15  6.09   1104900  ok
3    1546790400  5.75  6.11  6.14  5.69   8680300  ok
4    1546876800  5.15  5.81  5.81  5.13  14457600  ok
..          ...   ...   ...   ...   ...       ...  ..
237  1577289600  5.43  5.46  5.49  5.39   1401500  ok
238  1577376000  5.50  5.40  5.57  5.39   3109700  ok
239  1577635200  5.56  5.50  5.57  5.50   2534100  ok
240  1577721600  5.48  5.55  5.55  5.36   5974000  ok
241  1577894400  5.35  5.40  5.43  5.32   2379500  ok

[242 rows x 7 columns]
